# Visualizing annotation data

set up bokeh for notebook drawing, and import gene_viz

In [1]:
from bokeh.plotting import output_notebook, show
output_notebook(hide_banner=True)

import gene_viz
from gene_viz.features import Transcript, Exon, CDS

## Use PyEnsembl as the data source

**note** - requires [pyensembl](https://github.com/openvax/pyensembl)

In [2]:
import os
import pyensembl
from gene_viz.utils import transcripts_from_pyensembl

# set database cache directory for pyensembl
os.environ['PYENSEMBL_CACHE_DIR'] = os.path.abspath("./")

In [ ]:
# download and unzip gencode basic annotations for hg38
!wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_27/gencode.v27.basic.annotation.gtf.gz

In [3]:
# create pyensemble database from annotations
genome = pyensembl.Genome(
    reference_name="GRCh38",
    annotation_name="Gencode27",
    gtf_path_or_url="gencode.v27.basic.annotation.gtf.gz"
)
genome.index()

## Define the plot

In [4]:
def make_plot(transcripts):
    """
    Create a gene_plot object from a list of transcript feature objects
    """
    start = min([t.start for t in transcripts])
    end = max([t.end for t in transcripts])
    contig = next((t.contig for t in transcripts), "unlabelled")
    
    gene_plot = gene_viz.GenePlot(
        dict(
            row_height=15,
            pack=True,
            show_labels=True,
            label_horiz_position="center",
            label_vert_position="above",
            label_justify="center",
            label_offset=(0, -4),
            label_font_size="6pt",
            intron_width_percent=0.01,
            toolbar_location="above",
        )
    )
        
    gene_plot.x_range = (start, end)
    gene_plot.figure.xaxis.axis_label = contig
    gene_plot.transcripts = transcripts
    gene_plot.update()
    return gene_plot

## Query pyensembl and visualize the results

In [5]:
fig = make_plot(transcripts_from_pyensembl(genome, "chr22", 42124000, 42180000))
show(fig.figure)

# Do the same using GFFUtils as the data source

**note** - requires [GFFUtils](https://github.com/daler/gffutils)

In [6]:
import gffutils
import sqlite3

from gene_viz.utils import transcripts_from_gffutils

## Set up a GFFUtils database using the same annotation data

In [7]:
# note:
# database creation using gffutils is slower than pyensembl on the same dataset,
# so be patient
try:
    gffutils.create_db(
        "gencode.v27.basic.annotation.gtf.gz",
        "gencode.v27.basic.sqlite",
        keep_order=True,
        disable_infer_genes=True, disable_infer_transcripts=True)
except (sqlite3.OperationalError, AttributeError):
    pass

In [8]:
# connect
db = gffutils.FeatureDB("gencode.v27.basic.sqlite")

## Perform a query and visualize the results

In [9]:
fig = make_plot(transcripts_from_gffutils(db, "chr22", 42124000, 42180000))
show(fig.figure)